<a href="https://colab.research.google.com/github/Abhiram-108/CSE-LAB-303/blob/main/BackwardSelection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the dataset from the uploaded file
import pandas as pd

# Load the uploaded Melbourne housing dataset
file_path = '/content/melbourne_housing_raw.csv'
melbourne_data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
melbourne_data.head()


,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,2,h,NaN,SS,Jellis,3/9/16,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,2,h,1480000.0,S,Biggin,3/12/16,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,2,h,1035000.0,S,Biggin,4/2/16,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,3,u,NaN,VB,Rounds,4/2/16,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,3,h,1465000.0,SP,Biggin,4/3/17,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
import numpy as np

# Drop rows with missing target (Price)
melbourne_data_clean = melbourne_data.dropna(subset=['Price'])

# Select features and target
X = melbourne_data_clean.drop(['Price', 'Suburb', 'Date', 'CouncilArea', 'Regionname', 'SellerG', 'Type', 'Method'], axis=1)
y = melbourne_data_clean['Price']

# Handle missing values for the remaining numeric columns
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
model = RandomForestRegressor(random_state=42)

# Train the model with all features initially
model.fit(X_train, y_train)
y_pred_initial = model.predict(X_test)
mae_initial = mean_absolute_error(y_test, y_pred_initial)

# Perform backward feature elimination
mae_list = [mae_initial]
features = X_train.columns.tolist()

# Iterate over the number of features, removing the least important feature each time
# **Change:** Stop the loop before removing all features
for i in range(len(features) - 1):  # Stop one iteration earlier
    # Get feature importances
    model.fit(X_train, y_train)
    feature_importances = model.feature_importances_

    # Find the least important feature
    least_important_index = np.argmin(feature_importances)
    least_important_feature = features[least_important_index]

    # Remove the least important feature from the dataset
    X_train = X_train.drop(columns=[least_important_feature])
    X_test = X_test.drop(columns=[least_important_feature])

    # Retrain the model with the remaining features
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate the MAE and store it
    mae = mean_absolute_error(y_test, y_pred)
    mae_list.append(mae)

    # Remove the feature from the list
    features.remove(least_important_feature)

# Output the results
mae_list

[179476.4312564365,
 179550.99279074298,
 180086.93622940636,
 183336.16241149075,
 184117.91730353973,
 185062.34566818757,
 192390.0496203936,
 198422.50520947954,
 207475.50484132508,
 229791.3428620935,
 335183.8556700821,
 336796.30023891194]